In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
folder="/content/drive/MyDrive/Python/project241025"

In [8]:
!unzip "/content/drive/MyDrive/Python/project241025/60_100.zip"

Archive:  /content/drive/MyDrive/Python/project241025/60_100.zip
   creating: 60_100/100/
  inflating: 60_100/100/image001.png  
  inflating: 60_100/100/image007.png  
  inflating: 60_100/100/image008.png  
  inflating: 60_100/100/image009.png  
  inflating: 60_100/100/image024.png  
  inflating: 60_100/100/image025.png  
  inflating: 60_100/100/image026.png  
  inflating: 60_100/100/image027.png  
  inflating: 60_100/100/image028.png  
  inflating: 60_100/100/image029.png  
  inflating: 60_100/100/image030.png  
  inflating: 60_100/100/image031.png  
  inflating: 60_100/100/image032.png  
  inflating: 60_100/100/image034.png  
  inflating: 60_100/100/image035.png  
  inflating: 60_100/100/image036.png  
  inflating: 60_100/100/image037.png  
  inflating: 60_100/100/image038.png  
  inflating: 60_100/100/image039.png  
  inflating: 60_100/100/image040.png  
  inflating: 60_100/100/image041.png  
  inflating: 60_100/100/image042.png  
  inflating: 60_100/100/image043.png  
  inflating: 

In [9]:
import tensorflow as tf
import os, cv2, glob

images=[]
labels=[]
dict_labels = {"60":0, "100":1}
for folders in glob.glob("/content/60_100/*"):
    print(folders,"圖片讀取中…")
    for filename in os.listdir(folders):
        label=folders.split("/")[-1]
        try:
            img=cv2.imread(os.path.join(folders,filename))
            img = cv2.resize(img,dsize=(80,80))
            if img is not None:
                images.append(img)
                labels.append(dict_labels[label])
        except:
            pass
print("圖片讀取完畢!")

/content/60_100/60 圖片讀取中…
/content/60_100/100 圖片讀取中…
圖片讀取完畢!


In [10]:
print('圖片數量：{}'.format(len(images)))
print('標籤數量：{}'.format(len(labels)))

圖片數量：336
標籤數量：336


In [11]:
from sklearn.model_selection import train_test_split
import numpy as np
#from keras.utils import np_utils
from tensorflow.python.keras.utils import np_utils #新版語法

train_feature,test_feature,train_label,test_label = \
train_test_split(images, labels, test_size=0.2)

train_feature=np.array(train_feature)
test_feature=np.array(test_feature)
train_label=np.array(train_label)
test_label=np.array(test_label)

In [12]:
print('訓練資料維度：{}'.format(train_feature.shape))
print('訓練標籤維度：{}'.format(train_label.shape))

訓練資料維度：(268, 80, 80, 3)
訓練標籤維度：(268,)


In [13]:
print('測試資料維度：{}'.format(test_feature.shape))
print('測試標籤維度：{}'.format(test_label.shape))

測試資料維度：(68, 80, 80, 3)
測試標籤維度：(68,)


In [14]:
train_feature = train_feature/255
test_feature = test_feature/255

In [15]:
train_label = np_utils.to_categorical(train_label)
test_label = np_utils.to_categorical(test_label)

In [16]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Dropout, Flatten, Dense

model = Sequential()

In [17]:
model.add(Conv2D(filters=8, kernel_size=(5,5), padding='same',
                 input_shape=(80, 80, 3), activation='relu'))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [18]:
model.add(MaxPooling2D(pool_size=(2, 2)))

In [19]:
model.add(Dropout(0.2))

In [20]:
model.add(Conv2D(filters=16, kernel_size=(5,5),
                 padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

In [21]:
model.add(Conv2D(filters=32, kernel_size=(5,5),
                 padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

In [22]:
model.add(Flatten())
model.add(Dropout(0.2))

In [23]:
model.add(Dense(units=128, activation='relu'))

In [24]:
model.add(Dense(units=2,activation='softmax'))

In [25]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 80, 80, 8)           │             608 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 40, 40, 8)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 40, 40, 8)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 40, 40, 16)          │           3,216 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 20, 20, 16)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 20, 20, 16)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 20, 20, 32)          │          12,832 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 10, 10, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 10, 10, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 3200)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 3200)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │         409,728 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 2)                   │             258 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 426,642 (1.63 MB)

 Trainable params: 426,642 (1.63 MB)

 Non-trainable params: 0 (0.00 B)

In [26]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam', metrics=['accuracy'])
model.fit(x=train_feature, y=train_label, validation_split=0.2,
          epochs=20, batch_size=200, verbose=2)

Epoch 1/20
2/2 - 4s - 2s/step - accuracy: 0.3832 - loss: 0.9290 - val_accuracy: 0.6481 - val_loss: 0.6588
Epoch 2/20
2/2 - 2s - 1s/step - accuracy: 0.6449 - loss: 0.8702 - val_accuracy: 0.6481 - val_loss: 0.6690
Epoch 3/20
2/2 - 4s - 2s/step - accuracy: 0.6449 - loss: 0.6498 - val_accuracy: 0.6481 - val_loss: 0.6835
Epoch 4/20
2/2 - 2s - 1s/step - accuracy: 0.6402 - loss: 0.6767 - val_accuracy: 0.6481 - val_loss: 0.6868
Epoch 5/20
2/2 - 2s - 1s/step - accuracy: 0.6449 - loss: 0.6826 - val_accuracy: 0.6481 - val_loss: 0.6873
Epoch 6/20
2/2 - 2s - 791ms/step - accuracy: 0.6449 - loss: 0.6828 - val_accuracy: 0.6481 - val_loss: 0.6866
Epoch 7/20
2/2 - 3s - 1s/step - accuracy: 0.6402 - loss: 0.6813 - val_accuracy: 0.6481 - val_loss: 0.6857
Epoch 8/20
2/2 - 3s - 1s/step - accuracy: 0.6449 - loss: 0.6795 - val_accuracy: 0.6481 - val_loss: 0.6845
Epoch 9/20
2/2 - 3s - 1s/step - accuracy: 0.6449 - loss: 0.6780 - val_accuracy: 0.6481 - val_loss: 0.6829
Epoch 10/20
2/2 - 3s - 2s/step - accuracy: 

In [27]:
scores = model.evaluate(test_feature, test_label)
print('\n準確率=',scores[1])

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - accuracy: 0.5386 - loss: 0.6966 

準確率= 0.5147058963775635


In [28]:
import os, cv2, glob
from sklearn.model_selection import train_test_split
import numpy as np
#from keras.utils import np_utils
from tensorflow.python.keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Dropout, Flatten, Dense
images=[]
labels=[]
dict_labels = {"60":0, "100":1}
for folders in glob.glob("/content/60_100/*"):
    print(folders,"圖片讀取中…")
    for filename in os.listdir(folders):
        label=folders.split("/")[-1]
        try:
            img=cv2.imread(os.path.join(folders,filename))
            img = cv2.resize(img,dsize=(80,80))
            if img is not None:
                images.append(img)
                labels.append(dict_labels[label])
        except:
            pass
    print(folders,"圖片讀取完畢!")
train_feature,test_feature,train_label,test_label = \
train_test_split(images,labels,test_size=0.2)
train_feature=np.array(train_feature)
test_feature=np.array(test_feature)
train_label=np.array(train_label)
test_label=np.array(test_label)
train_feature = train_feature/255
test_feature = test_feature/255
train_label = np_utils.to_categorical(train_label)
test_label = np_utils.to_categorical(test_label)

model = Sequential()
model.add(Conv2D(filters=8, kernel_size=(5,5), padding='same',
                 input_shape=(80, 80, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
model.add(Conv2D(filters=16, kernel_size=(5,5),
                 padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
model.add(Conv2D(filters=32, kernel_size=(5,5),
                 padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dropout(0.2))
model.add(Dense(units=128, activation='relu'))
model.add(Dense(units=2,activation='softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer='adam', metrics=['accuracy'])
model.fit(x=train_feature, y=train_label,
          validation_split=0.2, epochs=20,
          batch_size=200, verbose=2)
scores = model.evaluate(test_feature, test_label)
print('\n準確率=',scores[1])
model.save('fan_model.h5')

/content/60_100/60 圖片讀取中…
/content/60_100/60 圖片讀取完畢!
/content/60_100/100 圖片讀取中…
/content/60_100/100 圖片讀取完畢!
Epoch 1/20
2/2 - 5s - 3s/step - accuracy: 0.3832 - loss: 0.8895 - val_accuracy: 0.6296 - val_loss: 0.6609
Epoch 2/20
2/2 - 2s - 818ms/step - accuracy: 0.6262 - loss: 0.6754 - val_accuracy: 0.6296 - val_loss: 0.6756
Epoch 3/20
2/2 - 3s - 1s/step - accuracy: 0.6262 - loss: 0.6635 - val_accuracy: 0.6296 - val_loss: 0.6822
Epoch 4/20
2/2 - 4s - 2s/step - accuracy: 0.6262 - loss: 0.6710 - val_accuracy: 0.6296 - val_loss: 0.6848
Epoch 5/20
2/2 - 4s - 2s/step - accuracy: 0.6262 - loss: 0.6769 - val_accuracy: 0.6296 - val_loss: 0.6849
Epoch 6/20
2/2 - 3s - 1s/step - accuracy: 0.6262 - loss: 0.6751 - val_accuracy: 0.6296 - val_loss: 0.6829
Epoch 7/20
2/2 - 3s - 1s/step - accuracy: 0.6262 - loss: 0.6724 - val_accuracy: 0.6296 - val_loss: 0.6800
Epoch 8/20
2/2 - 3s - 1s/step - accuracy: 0.6262 - loss: 0.6697 - val_accuracy: 0.6296 - val_loss: 0.6748
Epoch 9/20
2/2 - 4s - 2s/step - accuracy:


準確率= 0.5882353186607361


In [29]:
import numpy as np
import matplotlib.pyplot as plt
from keras.models import load_model
import glob,cv2

def show_images_labels_predictions(images, labels,
                  predictions,start_id, num=10):
    plt.figure(figsize=(12, 14))
    if num>25: num=25
    for i in range(0, num):
        ax=plt.subplot(5,5, 1+i)
        ax.imshow(images[start_id])
        if( len(predictions) > 0 ) :
            title = 'ai = ' + str(predictions[start_id])
            title += (' (o)' if predictions[start_id]== \
                      labels[start_id] else ' (x)')
            title += '\nlabel = ' + str(labels[start_id])
        else :
            title = 'label = ' + str(labels[start_id])
        ax.set_title(title,fontsize=12)
        ax.set_xticks([]);ax.set_yticks([])
        start_id+=1
    plt.show()


files = glob.glob("/content/*.jpg" )
#files = glob.glob("/content/drive/MyDrive/*.jpg" )
test_feature=[]
test_label=[]
dict_labels = {"60":0, "100":1}

for file in files:
    img=cv2.imread(file)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, dsize=(80,80))
    test_feature.append(img)
    label=file[10:13]
    test_label.append(dict_labels[label])
    #test_feature = np.array(test_feature)
    test_feature = np.array(test_feature).reshape(len(test_feature),40,40,3).astype('float32')
    test_label = np.array(test_label)

    test_feature_n = test_feature / 255 #有錯!!

try:

  model = load_model('/content/fan_model.h5')
  #prediction = tf.config.run_functions_eagerly(True)
  prediction = model.predict(test_feature_n)
  prediction = np.argmax(prediction,axis=1)
  show_images_labels_predictions(test_feature,test_label,prediction,0,len(test_feature))
except:
  print("模型未建立!")

模型未建立!


In [30]:
!pip install --upgrade gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.7/56.7 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.8/319.8 kB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.4/447.4 kB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.5/144.5 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 96.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/1

In [32]:
from tensorflow.keras.models import load_model
import gradio as gr

model = load_model("/content/fan_model.h5")

def fan60100(image):
    image = np.array(image.resize((80, 80))).astype("float32") / 255.0
    image = image.reshape(1, 80, 80, 3)
    prediction = model.predict(image).tolist()[0]
    class_names = ["60", "100"]
    return {class_names[i]: prediction[i] for i in range(2)}

inp = gr.Image(type="pil")
out = gr.Label(num_top_classes=2, label='預測結果')
grobj = gr.Interface(fn=fan60100, inputs=inp,
          outputs=out, title="圖片辨識")
grobj.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://aa7d000f34b1f6d0c2.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
